In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# First Create polyfeatures
df = pd.read_csv('L:/Python Projects All/UNZIP_FOR_NOTEBOOKS_FINAL/08-Linear-Regression-Models/Advertising.csv')
X = df.drop('sales',axis=1)
y = df['sales']
from sklearn.preprocessing import PolynomialFeatures
poly_converter = PolynomialFeatures(degree=3 , include_bias=False)
poly_features = poly_converter.fit_transform(X)
poly_features

array([[2.30100000e+02, 3.78000000e+01, 6.92000000e+01, ...,
        9.88757280e+04, 1.81010592e+05, 3.31373888e+05],
       [4.45000000e+01, 3.93000000e+01, 4.51000000e+01, ...,
        6.96564990e+04, 7.99365930e+04, 9.17338510e+04],
       [1.72000000e+01, 4.59000000e+01, 6.93000000e+01, ...,
        1.46001933e+05, 2.20434291e+05, 3.32812557e+05],
       ...,
       [1.77000000e+02, 9.30000000e+00, 6.40000000e+00, ...,
        5.53536000e+02, 3.80928000e+02, 2.62144000e+02],
       [2.83600000e+02, 4.20000000e+01, 6.62000000e+01, ...,
        1.16776800e+05, 1.84062480e+05, 2.90117528e+05],
       [2.32100000e+02, 8.60000000e+00, 8.70000000e+00, ...,
        6.43452000e+02, 6.50934000e+02, 6.58503000e+02]])

In [4]:
# Create splits
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(poly_features , y , test_size=0.3 , random_state=101)
# scale data
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
scalar.fit(x_train)
x_train_s = scalar.transform(x_train)
x_test_s = scalar.transform(x_test)

In [8]:
# Do ridge regularization like normal regression model
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha = 10)
ridge_model.fit(x_train_s , y_train)
predictions = ridge_model.predict(x_test_s)
# Now check error metrics
from sklearn.metrics import mean_absolute_error , mean_squared_error
mae = mean_absolute_error(y_test , predictions)
rmse = np.sqrt(mean_squared_error(y_test , predictions))
print(mae , rmse)

0.5774404204714183 0.8946386461319685


In [11]:
# Now check best value for alpha uff lambda with Cross Validation
from sklearn.linear_model import RidgeCV
ridge_model_cv = RidgeCV(alphas=(0.1 , 1.0 , 10.0 , 5.0 , 0.5 , 3.0 , 4.0))
# here it will choose leave one out cross validation to choose accurate alpha from the range I gave as cv - default value is leave one out
ridge_model_cv.fit(x_train_s , y_train)
predictions_cv = ridge_model_cv.predict(x_test_s)
mae = mean_absolute_error(y_test , predictions_cv)
rmse = np.sqrt(mean_squared_error(y_test , predictions_cv))
print(mae , rmse)

0.42737748843373746 0.6180719926921404


In [12]:
# Check best alpha value
print(ridge_model_cv.alpha_)

0.1


In [13]:
# now try with this alpha
ridge_model = Ridge(alpha = 0.1)
ridge_model.fit(x_train_s , y_train)
predictions = ridge_model.predict(x_test_s)
# Now check error metrics
from sklearn.metrics import mean_absolute_error , mean_squared_error
mae = mean_absolute_error(y_test , predictions)
rmse = np.sqrt(mean_squared_error(y_test , predictions))
print(mae , rmse)

0.42737748843288775 0.6180719926963003


In [15]:
# Now let's move to LASSO regularization
from sklearn.linear_model import LassoCV
lasso_cv_model = LassoCV(eps = 0.1 , n_alphas=100 )
lasso_cv_model.fit(x_train_s , y_train)
predictions = lasso_cv_model.predict(x_test_s)
mae = mean_absolute_error(y_test , predictions)
rmse = np.sqrt(mean_squared_error(y_test , predictions))
print(mae , rmse)
print(lasso_cv_model.alpha_)
print(lasso_cv_model.coef_)

0.6541723161252854 1.130800102276253
0.4943070909225828
[1.002651   0.         0.         0.         3.79745279 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [17]:
# Here I can check for more features resulting into a little less error
lasso_cv_model2 = LassoCV(eps = 0.001 , n_alphas=100 , max_iter=1000000)
lasso_cv_model2.fit(x_train_s , y_train)
predictions = lasso_cv_model2.predict(x_test_s)
mae = mean_absolute_error(y_test , predictions)
rmse = np.sqrt(mean_squared_error(y_test , predictions))
print(mae , rmse)
print(lasso_cv_model.alpha_)
print(lasso_cv_model.coef_)

0.43350346185900673 0.6063140748984039
0.4943070909225828
[1.002651   0.         0.         0.         3.79745279 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [20]:
# Here I can check for more features resulting into a little less error
lasso_cv_model2 = LassoCV(eps = 0.001 , n_alphas=100 , max_iter=6000000)
lasso_cv_model2.fit(x_train_s , y_train)
predictions = lasso_cv_model2.predict(x_test_s)
mae = mean_absolute_error(y_test , predictions)
rmse = np.sqrt(mean_squared_error(y_test , predictions))
print(mae , rmse)
print(lasso_cv_model.alpha_)
print(lasso_cv_model.coef_)

0.43350346185900673 0.6063140748984039
0.4943070909225828
[1.002651   0.         0.         0.         3.79745279 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.        ]


In [21]:
# Now we shall see Elastic Net - Combined L1 and L2
from sklearn.linear_model import ElasticNetCV
elastic_net_cv_model = ElasticNetCV(l1_ratio=[.1,.5,.7,.9,.95,.99,1] , eps = 0.001 , n_alphas=100 , max_iter=1000000)
elastic_net_cv_model.fit(x_test_s,y_test)
predictions = elastic_net_cv_model.predict(x_test_s)
mae = mean_absolute_error(y_test , predictions)
rmse = np.sqrt(mean_squared_error(y_test , predictions))
print(mae , rmse)
print(elastic_net_cv_model.alpha_)
print(elastic_net_cv_model.coef_)

0.39904655462963257 0.581200586366697
0.012268122349619192
[ 4.14333758  0.56260482 -0.         -2.45632727  3.3765281  -0.
  0.         -0.          0.         -0.          0.         -0.28702571
  0.         -0.         -0.          0.21014655 -0.          0.
  0.03960013]


In [22]:
print(elastic_net_cv_model.l1_ratio)

[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]


In [23]:
print(elastic_net_cv_model.l1_ratio_)

1.0
